#INSTALLATIONS

In [1]:
!pip install trl transformers accelerate datasets bitsandbytes torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

#INFERENCE with Pre-trained model

In [7]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [25]:
prompt = "In Yoga philosophy, what is the significance of the concept of ahimsa (non-violence)?"
model_input = tokenizer(prompt, return_tensors='pt')
model_input

{'input_ids': tensor([[151646,    641,  37573,  19128,     11,   1128,    374,    279,  25361,
            315,    279,   7286,    315,  22327,   5742,     64,    320,   6280,
          75727,    763,  11843]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [29]:
model_output = model.generate(model_input.input_ids, pad_token_id=tokenizer.eos_token_id)
model_response = tokenizer.decode(model_output[0], skip_special_tokens = True)
print(model_response)

In Yoga philosophy, what is the significance of the concept of ahimsa (non-violence)? How does this concept relate to the concept of non-violence in Indian law?

In the context of


#PREPARE DATA

In [3]:
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model

In [4]:
dataset = load_dataset("Abhaykoul/Ancient-Indian-Wisdom")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

dataset.json:   0%|          | 0.00/718k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/616 [00:00<?, ? examples/s]

In [5]:
dataset['train'][0]

{'output': 'Ahimsa, or non-violence, is a fundamental principle in Yoga philosophy. It teaches us to practice compassion and kindness towards all living beings, including ourselves. Ahimsa extends beyond refraining from physical harm and encompasses our thoughts, words, and actions. By cultivating ahimsa, we strive to create harmony and peace within and around us.\n\nThere\'s a beautiful parable about ahimsa from the Mahabharata, an ancient Indian epic. It\'s the story of a sage who, when asked to define ahimsa, replied, "When one regards all creatures as one\'s own self, and the pain and pleasure of all creatures as one\'s own, that is the highest state of ahimsa."\n\nBy embracing ahimsa, we contribute to a world where empathy, understanding, and love prevail. It\'s a powerful practice that fosters unity and respect for all forms of life.',
 'input': '',
 'instruction': 'In Yoga philosophy, what is the significance of the concept of "ahimsa" (non-violence)?'}

In [5]:
def format_dataset(examples):
    """Format the dataset into instruction-response pairs."""
    texts = []
    for instruction, response in zip(examples["instruction"], examples["output"]):
        # Combine instruction and response into a single text
        formatted_text = f"### Instruction:\n{instruction}\n\n### Response:\n{response}"
        texts.append(formatted_text)
    return {"text": texts}

# Apply formatting
dataset = dataset.map(format_dataset, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/616 [00:00<?, ? examples/s]

In [9]:
dataset['train'][0]

{'text': '### Instruction:\nIn Yoga philosophy, what is the significance of the concept of "ahimsa" (non-violence)?\n\n### Response:\nAhimsa, or non-violence, is a fundamental principle in Yoga philosophy. It teaches us to practice compassion and kindness towards all living beings, including ourselves. Ahimsa extends beyond refraining from physical harm and encompasses our thoughts, words, and actions. By cultivating ahimsa, we strive to create harmony and peace within and around us.\n\nThere\'s a beautiful parable about ahimsa from the Mahabharata, an ancient Indian epic. It\'s the story of a sage who, when asked to define ahimsa, replied, "When one regards all creatures as one\'s own self, and the pain and pleasure of all creatures as one\'s own, that is the highest state of ahimsa."\n\nBy embracing ahimsa, we contribute to a world where empathy, understanding, and love prevail. It\'s a powerful practice that fosters unity and respect for all forms of life.'}

# PREPARE for Training

In [8]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

In [9]:
peft_config = LoraConfig(
    r=16
    ,lora_alpha = 32
    ,lora_dropout = 0.1
    ,target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
    ,task_type = 'CAUSAL_LM'
)

In [10]:
model = get_peft_model(model, peft_config=peft_config)

In [12]:
training_args = TrainingArguments(
    output_dir="./results",         # Directory to save results
    num_train_epochs=20,            # Number of training epochs
    per_device_train_batch_size=4,  # Batch size per device
    per_device_eval_batch_size=4,   # Evaluation batch size
    gradient_accumulation_steps=4,  # Gradient accumulation steps
    gradient_checkpointing=False,   # Disable gradient checkpointing for debugging
    optim="adamw_torch",            # Optimizer
    learning_rate=1e-4,             # Learning rate
    warmup_ratio=0.1,               # Warmup ratio
    fp16=True,                      # Use mixed precision (FP16)
    logging_steps=10,               # Log every 10 steps
    save_strategy="steps",          # Save model at specific steps
    save_steps=100,                 # Save every 100 steps
    eval_strategy="steps",          # Evaluate at specific steps
    eval_steps=100,                 # Evaluate every 100 steps
    eval_accumulation_steps=1,      # Accumulate evaluation steps
    load_best_model_at_end=True,    # Load the best model at the end
    metric_for_best_model="eval_loss",  # Metric for best model
    greater_is_better=False,            # Lower eval_loss is better
    remove_unused_columns=True,         # Remove unused columns
    report_to="none",                   # Disable external logging
)

In [13]:
trainer = SFTTrainer(model=model,
                     args=training_args,
                     train_dataset = dataset["train"],
                     eval_dataset = dataset["train"].select(range(120))
                     )

Converting train dataset to ChatML:   0%|          | 0/616 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/616 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/616 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/616 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/120 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


#TRAIN!

In [14]:
trainer.train()

Step,Training Loss,Validation Loss
100,1.710500,1.752032


Step,Training Loss,Validation Loss
100,1.710500,1.752032
200,1.451100,1.545711
300,1.340200,1.412018
400,1.191400,1.326864
500,1.145800,1.266832
600,1.102600,1.225624
700,1.094000,1.202681


TrainOutput(global_step=760, training_loss=1.3429537961357518, metrics={'train_runtime': 1965.6828, 'train_samples_per_second': 6.268, 'train_steps_per_second': 0.387, 'total_flos': 3.5950886459904e+16, 'train_loss': 1.3429537961357518})

In [15]:
model.save_pretrained("fine_tuned_deepseek_llm")
tokenizer.save_pretrained("fine_tuned_deepseek_llm")

('fine_tuned_deepseek_llm/tokenizer_config.json',
 'fine_tuned_deepseek_llm/special_tokens_map.json',
 'fine_tuned_deepseek_llm/tokenizer.json')

# INFERENCE with Fine-Tuned Model

In [16]:
fine_tuned = "fine_tuned_deepseek_llm"
tokenizer = AutoTokenizer.from_pretrained(fine_tuned)
model = AutoModelForCausalLM.from_pretrained(fine_tuned)

In [19]:
prompt = "In Yoga philosophy, what is the significance of the concept of ahimsa (non-violence)?"
ftm_input = tokenizer(prompt, return_tensors='pt')
ftm_output = model.generate(ftm_input.input_ids, pad_token_id=tokenizer.eos_token_id)
ftm_response = tokenizer.decode(ftm_output[0], skip_special_tokens = True)
print(ftm_response)

In Yoga philosophy, what is the significance of the concept of ahimsa (non-violence)? How does this concept relate to ethical conduct in everyday life?

In the context of the Bhagavad


# EVALUATE with ROUGE

In [20]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1a550fd6acbca07fe5e0c14aa3ec0b99bcb26216b96eda88f501876b4d184c62
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [21]:
from rouge_score import rouge_scorer

In [22]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [30]:
scores = scorer.score(model_response, ftm_response)
for key in scores:
    print(f'{key}: {scores[key]}')

rouge1: Score(precision=0.8387096774193549, recall=0.8125, fmeasure=0.8253968253968254)
rougeL: Score(precision=0.8064516129032258, recall=0.78125, fmeasure=0.7936507936507936)
